## <span style = "font-family: Cambria">**Task: Classification model to accurately distinguish benign and malignant tumours**</span>

Here are some of the references that we used when approaching this task:
1. Nguyen, A., Yosinski, J., & Clune, J. (2015). Deep neural networks are easily fooled: High confidence predictions for unrecognizable images. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 427-436).

2. Anaya-Isaza, A., Mera-Jiménez, L., Verdugo-Alejo, L., & Sarasti, L. (2023). Optimizing MRI-based brain tumour classification and detection using AI: A comparative analysis of neural networks, transfer learning, data augmentation, and the cross-transformer network. European journal of radiology open, 10, 100484. https://doi.org/10.1016/j.ejro.2023.100484

3. Arsa, Dewa Made Sri, and Anak Agung Ngurah Hary Susila. “VGG16 in Batik Classification Based on Random Forest.” In 2019 International Conference on Information Management and Technology (ICIMTech), 295–99. Jakarta/Bali, Indonesia: IEEE, 2019. https://doi.org/10.1109/ICIMTech.2019.8843844.

4. Selvaraju, Ramprasaath R., Michael Cogswell, Abhishek Das, Ramakrishna Vedantam, Devi Parikh, and Dhruv Batra. “Grad-CAM: Visual Explanations from Deep Networks via Gradient-Based Localization.” In 2017 IEEE International Conference on Computer Vision (ICCV), 618–26. Venice: IEEE, 2017. https://doi.org/10.1109/ICCV.2017.74.



Some limitations we wish to resolve:
1. Nguyen et al. (2015) touched on the unpredictable behaviour of AlexNet. However, due to the blackbox nature of CNN, we could explain the rationale behind the unpredictable nature. Hence, we wish to improve on the interpretability of CNN in image classifications through the use of GradCAM.
2. Anaya-Isaza et al. explored geometric flipping image augmentation in their work. We wish to extend and further explore other image augmentation techniques with Keras ImageDataGenerator to improve robustness of training with limited datasets.
3. Arsa et al. explored the use of VGG16-Random Forest for Batik Classification, we wish to extend on it by exploring the CNN-XGBoost pairing in our image classification due to the strong regularisation potential of XGBoost.

### <span style = "font-family: Cambria">Content explorer</span>

In order, this notebook is categorised into:
1. [Helper functions](#helper-functions)
- General functions i.e., plotting , image augmentation
- Preprocessing functions
- GradCAM Helper functions
2. [Data Preparation](#prepare)
3. [Baseline model](#baseline)
4. [VGG16-ANN model](#ann)
5. [VGG16-XgBoost model](#xgboost)

### <span style = "font-family: Cambria">Installing dependencies and imports</span>

In [ ]:
#Uncomment and run to install relevant packages
#Note that this notebook may not run for newer keras versions
'''
!pip install numpy==1.24.3
!pip install pandas==1.5.3
!pip install matplotlib==3.6.3
!pip install imutils==0.5.4
!pip install opencv-python==4.9.0.80
!pip install ImageHash==4.3.1
!pip install tqdm==4.65.0
!pip install scikit-learn==1.4.1.post1
!pip install scikit-image==0.22.0
!pip install xgboost==2.0.3
!pip install scikeras==0.12.0
!pip install keras==2.12.0
!pip install tensorflow==2.12.1
'''

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import imutils
import cv2
import os
import random
import pickle
import imagehash
from PIL import Image
from tqdm import tqdm

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from skimage import color
from xgboost import XGBoost
from scikeras.wrappers import KerasClassifier


import keras
from keras.applications import VGG16
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation, Flatten, Dropout, Conv2D, MaxPool2D, BatchNormalization, MaxPooling2D
from keras.optimizers import Adam, SGD

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, AUC

import sklearn.metrics
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, f1_score, accuracy_score, precision_score, recall_score

# <a id = "helper-functions"></a> <span style = "font-family: Cambria">**Helper functions**</span>

## <span style = "font-family: Cambria">General functions</span>

### <span style = "font-family: Cambria">1. Finding images from filesystem</span>

In [ ]:
def fileList(source):
    '''
    Returns list of filepaths with .jpg extensions from root
    '''
    matches = []
    for root, _, filenames in os.walk(source):
        for filename in filenames:
            if filename.endswith((".jpg")):
                matches.append(os.path.join(root, filename))
    return matches

### <span style = "font-family: Cambria">2. Plot helper functions</span>

In [ ]:
def grid_plot(images, dim = (10,10)):
    '''
    Display the images in a grid.

    Args:
        image (nd.array): Input image (numpy array).
    '''
    rows, cols = dim
    r_c, c_c = 0, 0
    fig = plt.figure(figsize=(1.7*rows, 1.3*cols))

    #Looping over for i = rows x cols
    for i, image in enumerate(images[:(rows*cols)]):
        fig.add_subplot(rows, cols, i+1)
        plt.imshow(image)
        plt.tight_layout()
        plt.axis(False)

        r_c+=1
        c_c+=1

    plt.subplots_adjust(hspace=0, wspace=0)
    plt.show()

### <span style = "font-family: Cambria">3. Shuffling 2 arrays</span>

In [ ]:
def shuffling(a, b):
    """
    Shuffle 2 numpy arrays in the same order

    Args:
        a (nd.array)
        b (nd.array)
    """
    p = np.random.permutation(len(a))
    return a[p], b[p]

### <span style = "font-family: Cambria">4. Image augmentation</span>

In [ ]:
train_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="nearest")

validation_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True)

# particularly for GridSearchCV since we cannot pass in a generator object
def augment_images(X):
    '''
    Apply augmentation strategies to images.

    Args:
        X (nd.array): Input data of image arrays

    Returns:
        new_imgs (nd.array): Numpy array of augmented images
    '''

    new_imgs = []
    for image in X:
        imba_augmenter = ImageDataGenerator()

        # augment the images to make models robust
        augment_param = {
                "theta": random.randint(-40, 40),
                "shear": random.uniform(-0.2, 0.2),
                "tx": random.uniform(-0.2, 0.2),
                "ty": random.uniform(-0.2, 0.2),
                "zx": random.uniform(-0.2, 0.2),
                "zy": random.uniform(-0.2, 0.2),
                "flip_horizontal": True,
                "flip_vertical": bool(random.randint(0, 1)),
            }

        new_img = imba_augmenter.apply_transform(image, augment_param)

        # standardise the data
        new_img = imba_augmenter.standardize(new_img)
        new_imgs.append(new_img)

    return np.array(new_imgs)

### <span style = "font-family: Cambria">5. Initialising VGG16 TF model</span>

In [ ]:
def create_vgg16tf():
    """
    Creates a VGG16 transfer learning model with a customer classifier

    Returns:
        model (keras.Model)
    """

    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = VGG16(input_tensor=inputs ,weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    x.trainable = False

    x = Flatten()(x.output)
    x = Dense(256, activation="relu")(x)
    x = Dropout(rate=0.4)(x)
    x = Dense(2, activation="softmax")(x)
    model = tf.keras.Model(inputs, x)

    return model

create_vgg16tf().summary()

### <span style = "font-family: Cambria">6. Loading model with best weights</span>

In [ ]:
def load_best_model(path, create_model_fn):
    """
    Load model with best weights.

    Args:
        path (str): Filepath of .h5 file containing the best weights.
        create_model_fn (function): Function to create empty model, should be same model architecture as weights file
    """

    model = create_model_fn()
    model.load_weights(path)

    return model

### <span style = "font-family: Cambria">7. Additional metrics for VGG16-ANN model</span>

In [ ]:
def added_metrics(pred_prob, grd_truth):
    """
    Custom metrics function for VGG16-ANN

    Args:
        pred_prob (nd.array): Predictions of model (n x 2)
        grd_truth (nd.array): Labels (n x 2)

    Returns:
        (list) List of metrics
    """
    pred = np.argmax(pred_prob, axis = 1)
    grd_truth = np.argmax(grd_truth, axis = 1)
    tn, fp, fn, tp = confusion_matrix(pred, grd_truth).ravel()

    accuracy = (tp+tn)/(fp+fn+tp+tn)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)

    return [tn, tp, fn, fp, precision, accuracy, recall]


## <a id = "preprocess"></a><span style = "font-family: Cambria">Dataset processing functions</span>

### <span style = "font-family: Cambria">1. Removing near duplicates</span>

In [ ]:
def display_images_in_row(image_paths, labels=None, size=(5, 5)):
    """
      Plot images in a row

      Args:
          image_paths (list): List of image paths
          labels (list): Title for plots
          size (tuple): Size of figure
    """
    num_images = len(image_paths)
    if num_images == 0:
        return

    fig, axes = plt.subplots(1, num_images, figsize=(num_images * size[0]/3, size[1]))

    for i, (image_path, label) in enumerate(zip(image_paths, labels)):
        image = Image.open(image_path)
        axes[i].imshow(image)
        axes[i].axis('off')
        if labels:
            axes[i].set_title(label)

    plt.show()

def find_near_duplicates(image_paths, threshold=5, verbose=0):
      """
      Utilises phash from imagehash library to remove duplicates and near duplicates from dataset.
      Shows number of duplicated and unique images from each filepaths

      Args:
          image_paths (list): List of image paths
          threshold (int or float): Threshold used to determine near duplicates (minimum hamming difference
                                    of image hashs to be considered unique)
          verbose (int): 0 - No plots
                         1 - Show rows of duplicated images

      Returns:
          unique_image_paths (list): List of unique image paths
      """

      # Dictionary to store hashes for each image
      image_hashes = {}
      # List to store near-duplicate pairs
      near_dups = {}
      # Iterate over all image paths
      for image_path in image_paths:
          # Load image
          image = Image.open(image_path).convert('L')  # Convert to grayscale
          # Calculate perceptual hash
          image_hash = imagehash.phash(image)
          # Store hash in dictionary
          image_hashes[image_path] = image_hash
      # Compare hashes to find near-duplicates
      for path1, hash1 in image_hashes.items():
        #if dup already considered avoid overlap
        if path1 in near_dups:
          continue
        list_of_dups = []
        dup_image_names=[]
        for path2, hash2 in image_hashes.items():

          # If images are not the same and if the hamming distance of 2 images is less than threshold
          # This parts tracks the duplicated images
          if path1 != path2 and hash1 - hash2 < threshold:
            near_dups[path2]=hash2
            if len(list_of_dups) ==0:
              list_of_dups.append(path1)
              dup_image_names.append(os.path.basename(path1))
            list_of_dups.append(path2)
            dup_image_names.append(os.path.basename(path2))

        if verbose == 1:
          if len(list_of_dups) >0:
            print(os.path.dirname(list_of_dups[0]))
            print(dup_image_names)

      unique_image_paths = []
      tumour_count = 0

      for path in image_paths:
        if path not in near_dups:
          unique_image_paths.append(path)
          tumour_count += 1

      print(f"Number of duplicated {os.path.basename(os.path.dirname(path))} tumours: {len(near_dups)}")
      print(f"Number of {os.path.basename(os.path.dirname(path))} tumours: {tumour_count}")
      return unique_image_paths

### <span style = "font-family: Cambria">2. Crop image</span>

In [ ]:
def crop_img(img):
    """
    Crops out majority of the background and centralised the brain

    Args:
        img (nd.array)

    Returns:
        new_imgs(nd.array): Cropped image resized to 224 pixels by 224 pixels
    """

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Clearing noises
    thresh = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Finding contours and grab the main image
    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    c = max(contours, key = cv2.contourArea)

    # Finding boundaries
    left = tuple(c[c[:, :, 0].argmin()][0])
    right = tuple(c[c[:, :, 0].argmax()][0])
    top = tuple(c[c[:, :, 1].argmin()][0])
    bottom = tuple(c[c[:, :, 1].argmax()][0])

    # New image cropping
    new_img = img[top[1] : bottom[1], left[0] : right[0]]
    new_img = cv2.resize(new_img, (224,224))

    return new_img

### <span style = "font-family: Cambria">3. Image mask</span>

In [ ]:
def mask(image):
    '''

    Create a binary mask separating the brain from the background, postprocess and apply the brain mask to original.

    Args:
        image (nd.array): Input image (numpy array).

    Returns:
        masked_image (nd.array): Image with the mask applied.
    '''

    # Apply thresholding to create a binary mask separating the brain from the background
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray_image, (7,7), 0)
    normalized = cv2.normalize(blur, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    _, thresh = cv2.threshold(normalized, 10, 255, cv2.THRESH_BINARY)
    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Postprocessing
    contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    brain_mask = np.zeros_like(normalized)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 500:  # threshold
            cv2.drawContours(brain_mask, [cnt], -1, 255, -1)

    # Apply the brain mask to the original image
    masked_brain = cv2.bitwise_and(image, image, mask=brain_mask)

    return masked_brain

### <span style = "font-family: Cambria">4. Image Augmentation for class imbalance</span>

In [ ]:
def augment_minority_class(train_X, train_y, verbose=0, seed=42):
    '''
    Augment only benign images in train X and from there, apppend to train X and y.

    Args:
        train_X (nd.array): Input image (numpy array).
        train_y (nd.array): Class labels (numpy array).

    Returns:
        train_X (nd.array): Input image with more benign images.
        train_y (nd.array): Class labels with more 0s.
    '''

    random.seed(seed)
    _, counts = np.unique(train_y, return_counts = True)
    min_class = np.argmin(counts)

    augment_count = max((max(counts)//min(counts)) - 1, 1) * min(counts)

    # Separate benign images in train_X
    min_class_idx = np.argwhere(train_y==min_class).reshape(1,-1)[0]
    train_X_minclass = train_X[min_class_idx]

    # Generator seed provides seeds for image augmentation based on parent seed
    generator_seed = [random.randint(0,10000) for _ in range(augment_count)]
    imba_augmenter = ImageDataGenerator()

    new_imgs=[]
    for count, img in enumerate(train_X_minclass):
        if count >= augment_count:
            break

        random.seed(generator_seed[count])

        ### EDIT AUGMENT PARAMETERS HERE
        augment_param = {
            "theta": random.randint(-35, 35),
            "shear": random.uniform(0, 0.1),
            "tx": random.uniform(0.8, 1.3),
            "ty": random.uniform(0.8, 1.3),
            "zx": random.uniform(0.8, 1.3),
            "zy": random.uniform(0.8, 1.3),
            "flip_horizontal": True,
            "flip_vertical": bool(random.randint(0, 1)),
        }

        new_img = imba_augmenter.apply_transform(img, augment_param)
        new_imgs.append(new_img)

        # Verbose of 1 allows to see how is the augmentation like
        if verbose == 1:
            print(augment_param)
            plt.imshow(new_img)
            plt.show()

        train_X = np.append(train_X, new_img[np.newaxis,:,:,:], axis=0)
        train_y = np.append(train_y, min_class)
        train_X, train_y = shuffling(train_X, train_y)

    # Verbose of 2 provides grid plot of augmented images
    if verbose == 2:
        grid_plot(new_imgs)

    return train_X, train_y

#USAGE: augmented_images = augment_minority_class(train_X, train_y, verbose=1)


def augment_images_xgboost(input, num_of_images, verbose=0, seed=42):
    '''
    Slightly different augment function. Able to augment benign and malignant class separately
    with stated number of images

    Args:
        input (nd.array): Input image (numpy array).
        num_of_images (int): Number of times to augment

    Returns:
        new_imgs (nd.array): Input image with more benign images.
    '''

    random.seed(seed)
    if not isinstance(input[0], np.ndarray) or not len(input[0].shape) >= 2:
        input = [cv2.imread(path) for path in input]

    # Generator seed provides seeds for image augmentation based on parent seed
    generator_seed = [random.randint(0,10000) for _ in range(num_of_images)]
    imba_augmenter = ImageDataGenerator()

    new_imgs = []
    for count in range(num_of_images):
        if count < len(input):
            index = count
        else:
            index = random.randint(0, len(input)-1)

        img = input[index]

        random.seed(generator_seed[count])

        ### EDIT AUGMENT PARAMETERS HERE
        augment_param = {
            "theta": random.randint(-35,35),
            "shear": random.uniform(0, 0.1),
            "zx": random.uniform(0.8, 1.3),
            "zy": random.uniform(0.8, 1.3),
            "flip_horizontal": True,
            "flip_vertical": bool(random.randint(0, 1))
        }

        new_img = imba_augmenter.apply_transform(img, augment_param)

        # Verbose of 1 allows to see how is the augmentation like
        if verbose == 1:
            print(augment_param)
            plt.imshow(new_img)
            plt.show()
        new_imgs.append(new_img)
    return np.array(new_imgs)

### <span style = "font-family: Cambria">5. Master preprocessing pipeline</span>

In [ ]:
def preprocessing(img_paths, verbose=0):
    '''
    Read the images from image paths and preprocess images using all the above preprocessing functions (1-4).

    Args:
        img_paths (List): List of file paths

    Returns:
        processed (List): List of numpy arrays of unique images cropped and masked.
    '''
    img_paths_no_dup = find_near_duplicates(img_paths, verbose=verbose)
    img_list = [cv2.imread(path) for path in img_paths_no_dup]

    processed = []
    for img in img_list:
        img = mask(img)
        processed.append(crop_img(img))

    return processed


## <a id = "explain"></a><span style = "font-family: Cambria">GradCAM helper function</span>

In [ ]:
#Create gradcam model with input image connected with output of last pooling layer and output

def grad_heatmap(model, img_bg_data, img_bg_label , layer_name_for_grad_cal, dim=(7,10)):
    """
    Provides plots of GradCAM heatmap visualisations using grid_plot
    Code for computing GradCAM heatmap and visualising it is obtained from: https://keras.io/examples/vision/grad_cam/

    GradCAM algorithm referenced from
        Selvaraju, Ramprasaath R., Michael Cogswell, Abhishek Das, Ramakrishna Vedantam, Devi Parikh,
        and Dhruv Batra. “Grad-CAM: Visual Explanations from Deep Networks via Gradient-Based Localization.”
        In 2017 IEEE International Conference on Computer Vision (ICCV), 618–26.
        Venice: IEEE, 2017. https://doi.org/10.1109/ICCV.2017.74.

    Args:
        model (keras.Model)
        img_bg_data (List): List of image arrays (rank 3 numpy arrays)
        img_bg_label (nd.array): numpy array (rank 2) e.g. [[0 1][1 0][1 0]]
        layer_name_for_grad_cal (str): Ideally layer.name of last convolution/ pooling layer

        dim (tuple): Gradcam will run through top (r x c) and output in a grid plot
    """

    grad_model = keras.Model(
        model.inputs, [model.get_layer(layer_name_for_grad_cal).output, model.output]
    )

    #Initializing matplotlib parameters
    alpha = 0.4
    rows, cols = dim
    r_c, c_c = 0, 0
    fig = plt.figure(figsize=(24, 16))

    #Looping over for i = rows x cols
    for i, img in tqdm(enumerate(img_bg_data[:(rows*cols)])):
        fig.add_subplot(rows, cols, i+1)

        #Initialising train images for gradcam from img_bg_data
        img_array = np.expand_dims(img, axis=0)

        #Here onwards ref: https://keras.io/examples/vision/grad_cam/
        #Track gradient change in grad_model with img_array
        with tf.GradientTape() as tape:
            last_conv_layer_output, preds = grad_model(img_array)
            pred_index = tf.argmax(preds, axis = -1)
            class_channel = preds[:,int(pred_index.numpy())]

        # Initialising labels (segregating for 1 or 2 output models)
        if len(img_bg_label.shape) != 1:
            model_pred = pred_index.numpy()
            grd_truth = np.argmax(img_bg_label,axis=1)[i]
        else:
            model_pred = int(tf.cast(preds>0.5, tf.int32).numpy())
            grd_truth = img_bg_label[i]

        # Gradient of output with respect to last convolution layer
        grads = tape.gradient(class_channel, last_conv_layer_output)

        # This is a vector where each entry is the mean intensity of the gradient
        # over a specific feature map channel
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

        # We multiply each channel in the feature map array
        # by "how important this channel is" with regard to the top predicted class
        # then sum all the channels to obtain the heatmap class activation
        last_conv_layer_output = last_conv_layer_output[0]
        heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]

        heatmap = tf.squeeze(heatmap)
        heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
        heatmap = np.uint8(255 * heatmap)

        # Use jet colormap to colorize heatmap
        jet = mpl.colormaps["jet"]
        jet_colors = jet(np.arange(256))[:, :3]
        jet_heatmap = jet_colors[heatmap]

        # Create an image with RGB colorized heatmap
        jet_heatmap = keras.utils.array_to_img(jet_heatmap)
        jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
        jet_heatmap = keras.utils.img_to_array(jet_heatmap)

        # Superimpose the heatmap on original image
        superimposed_img = jet_heatmap * alpha + img
        superimposed_img = keras.utils.array_to_img(superimposed_img)

        # Display Grad CAM
        plt.imshow(superimposed_img)

        if int(model_pred) != int(grd_truth):
            plt.title(f"id:{i}; Pred {int(model_pred)}; Truth {int(grd_truth)}", color="red")
        else:
            plt.title(f"id:{i}; Pred {int(model_pred)}; Truth {int(grd_truth)}")

        plt.tight_layout()
        plt.axis(False)

        r_c+=1
        c_c+=1

    plt.subplots_adjust(wspace=0)
    plt.show()
    return

# <a id = "prepare"></a> <span style = "font-family: Cambria">**Data Preparation**</span>
This part consists of:
1. Loading and preparing unique images
2. Splitting dataset into training and test data

In [ ]:
# load data
b_dataset = Path("./Dataset/Image Dataset/Brain Tumor/data/benign")
m_dataset = Path("./Dataset/Image Dataset/Brain Tumor/data/malignant")

b_dataList = fileList(b_dataset)
m_dataList = fileList(m_dataset)

#Excluding outlier non-brain MRI image
outlier = "Dataset\\Image Dataset\\Brain Tumor\\data\\malignant\\248.jpg"
if outlier in m_dataList:
    print("Removing outlier")
    plt.imshow(cv2.imread(outlier))
    plt.show()
    m_dataList.remove(outlier)

# preprocess images
b_imgs = preprocessing(b_dataList)
m_imgs = preprocessing(m_dataList)

In [ ]:
# Visualise pre-processed images
grid_plot(b_imgs)
grid_plot(m_imgs)

### <span style = "font-family: Cambria">Split data into training and testing data</span>

In [ ]:
# Initialising labels
b_label = [0 for _ in range(len(b_imgs))]
m_label = [1 for _ in range(len(m_imgs))]

# Mixing order of class 0 and class 1
t_imgs = b_imgs + m_imgs
t_label = b_label + m_label
t_List = list(zip(t_imgs, t_label))

t_imgs, t_label = map(list,(zip(*t_List)))
X, test_X, y, test_y = train_test_split(t_imgs, t_label, train_size=0.8, shuffle=True, random_state=42)

X = np.array(X)
y = np.array(y)

print(f"Train: {len(y)} Test: {len(test_y)}")

# <a id = "baseline"></a> <span style = "font-family: Cambria">**Baseline model**</span>

In [ ]:
# Additional variable changes to adapt to baseline model
y_base = y.astype(np.float32)
test_X_base = np.array(test_X)
test_y_base = np.array(test_y).astype(np.float32)
img_width, img_height = 224, 224

## <span style = "font-family: Cambria">Baseline model creation</span>

In [ ]:
# Evaluation metrics
METRICS = ["AUC", "binary_accuracy", "TruePositives", "TrueNegatives", "FalsePositives", "FalseNegatives", "Precision", "Recall", "F1Score"]

# A function to create baseline model
def create_baseline_model(learning_rate = 0.001, optimizer = SGD, num_layers = 2, dropout_rate = 0.4):
    '''
    Create baseline model for hyperparameter tuning or classification.

    Args:
        learning_rate (float): The learning rate in model compile.
        optimizer (keras.optimizer): Optimizer of choice.
        num_layers (int): Number of convolutional layers expanded after the first 32-filter Conv layer
        dropout_rate (float): The probability of dropping out each unit in the hidden layer to prevent overfitting.

    Returns:
        baseline_model: The model created with default / best hyperparameters.
    '''

    baseline_model = Sequential()
    baseline_model.add(Input(shape = (img_width, img_height, 3)))

    # A single 32-filter Conv layer to extract low-level features i.e., contours
    baseline_model.add(Conv2D(filters=32, kernel_size=3, activation='relu', padding='valid'))
    baseline_model.add(BatchNormalization()) # to normalise ReLU activation function to speed up and stabilise training
    baseline_model.add(MaxPool2D(pool_size=(2, 2))) # Condense feature map

    # Expand Conv layers to capture increasingly complex features
    for _ in range(num_layers):
        baseline_model.add(Conv2D(64 * (_ + 1), kernel_size=3, activation='relu', padding = "valid"))
        baseline_model.add(MaxPool2D((2, 2))) # Condense feature map

    # Flatten 3D output into 1D array before feeding into a fully connected layer
    baseline_model.add(Flatten())
    baseline_model.add(Dense(256, activation='relu')) # Extract features
    baseline_model.add(Dropout(dropout_rate)) # prevent overfitting
    baseline_model.add(Dense(1, activation='sigmoid')) # sigmoid fn to compute probability that the image belongs to malignant class

    # optimizer used to adjust weights and learning rates by minimising loss
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = SGD(learning_rate=learning_rate)

    # compile model against binary cross entropy LF and evaluation metrics
    baseline_model.compile(optimizer= opt, loss=keras.losses.binary_crossentropy, metrics=METRICS)

    return baseline_model

create_baseline_model.summary()

## <span style = "font-family: Cambria">Finetuning hyperparameters</span>

In [ ]:
# To solve kernel death when running GridSearchCV
os.environ['KMP_DUPLICATE_LIB_OK'] = "True"

# Initialise the combinations of hyperparameter
param_grid = {
    'model__learning_rate': [0.0001, 0.001, 0.01],
    'model__optimizer': ['adam', 'sgd'],
    'model__num_layers': [2, 3, 4],
    'model__dropout_rate': [0.3, 0.4, 0.5],
    'batch_size': [32, 64, 128]
}

keras_wrapper = KerasClassifier(build_fn=create_baseline_model, verbose = 0)

# StratifiedKFold CV to maintain class distribution
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=keras_wrapper, param_grid=param_grid, cv=skf, scoring='roc_auc', n_jobs= 6)

# Upsample minority class
X, y = augment_minority_class(X, y)

# Augment images to make model more robust
X = augment_images(X)

grid_search.fit(X, y)

# find best parameters
print("Best hyperparameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

best_model = grid_search.best_estimator_
test_score = best_model.score(test_X, test_y)
print("Test AUC:", test_score) # the score in this output is low, but this run may be an artefact as evident in the next cell

### <span style = "font-family: Cambria">Feeding best hyperparameters from GridSearchCV to get fine-tuned model</span>

In [ ]:
# hyperparameters
num_folds = 5
epochs = 10
batch_size = 64
best_val_loss = 1000
filepath="baseline_model.weights.h5"

# from GridSearchCV
dropout_rate = 0.4
learning_rate = 0.01
num_layers = 3
optimizer = SGD

skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

scores = {}
for fold_idx, (train_idx, val_idx) in tqdm(enumerate(skf.split(X, y))):

    # Reinitialising model
    train_X, val_X = X[train_idx], X[val_idx]
    train_y, val_y = y[train_idx], y[val_idx]

    # augment the minority (benign) class
    train_X, train_y = augment_minority_class(train_X, train_y)

    # increase dataset and standardise trainig data
    train_generator = train_datagen.flow(
            x=train_X,
            y=train_y,
            batch_size=batch_size)

    # standardise validation data
    val_generator = validation_datagen.flow(
        x=val_X,
        y=val_y,
        batch_size = batch_size
    )

    # clear previous iteration and reinitialise model
    keras.backend.clear_session()
    baseline_model = create_baseline_model(num_layers=num_layers, dropout_rate=dropout_rate, learning_rate=learning_rate, optimizer=optimizer)

    baseline_history = baseline_model.fit(train_generator,
                        epochs=epochs,
                        validation_data = val_generator)

    if baseline_history.history["val_loss"][-1] <= best_val_loss:
        best_val_loss = baseline_history.history["val_loss"][-1] #replace
        baseline_model.save_weights(filepath)

    scores["loss"], scores["AUC"], scores["binary_accuracy"], scores["TP"], scores["TN"], scores["FP"], scores["FN"], scores["precision"], scores["recall"],  scores["f1_score"] = baseline_model.evaluate(val_generator)

    print(f"Fold {fold_idx + 1}: {scores}")

# load best weights
baseline_model = create_baseline_model(num_layers=num_layers, dropout_rate=dropout_rate, learning_rate=learning_rate, optimizer=optimizer)
baseline_model.load_weights(filepath)

# preprocessing test data
test_generator = validation_datagen.flow(
        x=test_X,
        y=test_y
)

scores["loss"], scores["AUC"], scores["binary_accuracy"], scores["TP"], scores["TN"], scores["FP"], scores["FN"], scores["precision"], scores["recall"],  scores["f1_score"] = baseline_model.evaluate(test_generator)
print(f"Performance metrics: {scores}")

### <span style = "font-family: Cambria">Plot AUC and loss graphs</span>

In [ ]:

# Plot training & validation AUC values
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(baseline_history.history['auc'], color = "midnightblue")
plt.plot(baseline_history.history['val_auc'], color = "salmon")
plt.ylabel('AUC of Baseline CNN')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(baseline_history.history['loss'], color = "midnightblue")
plt.plot(baseline_history.history['val_loss'], color = "salmon")
plt.ylabel('Loss of Baseline CNN')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='center right')

plt.tight_layout()
plt.savefig("model_metrics.jpg", dpi = 250)


### <span style = "font-family: Cambria">Gradcam</span>

In [ ]:
grad_heatmap(baseline_model, X_aug, y_aug, "conv2d_5", dim=(7,10))

In [ ]:
grad_heatmap(baseline_model, test_X_base, test_y_base, "conv2d_5", dim=(4,10))

# <a id = "ann"></a><span style = "font-family: Cambria">**VGG16-ANN model**</span>

### <span style = "font-family: Cambria">Transfer learning experiment</span>

In [ ]:
# Will be looping through this to unfreeze layers from the back for training
# during finetuning

layers_frozen = [2, 3, 8, 9, 10, 12, 13, 14, 16, 17]

"""
Note: Runtime for this cell is very long

Aim: To understand the effects of fine-tuning from different convolution layers of VGG16,
we first trained the custom classifier for 10 epochs, before freezing the first n layers
and training the rest of the convolutional layers at a lower learning rate.

Algorithm flow:

Outer loop around StratifiedKFold(n_split = 5): {

    Train base VGG16-ANN model while only training custom classifier at back of model
    Save base model weights

    Inner loop around layers_frozen: {
        Load base model weights
        Run through layers_frozen to unfreeze selected layers
        Finetune model for all the unfrozen layers
        Save new model weights
    }

}

Refer to the next segment "Transfer learning exp summary" for summary of results
"""
batch_size = 64
epochs = 10
epochs_finetune = 10

finetune_means = []
finetune_errors = []

best_val_loss = 10000

# To record performance across layers_frozen loops
layer_metrics = {l:[] for l in layers_frozen}


# Running experiment across stratifiedkfold
skf = StratifiedKFold(n_splits=5)
for fold_idx, (train_idx, val_idx) in tqdm(enumerate(skf.split(X, y))):

    fold_filepath = f"./tf_exp/vggbasemodel_{fold_idx}.h5"
    model = create_vgg16tf()

    # Segregating train and val
    train_X, val_X = X[train_idx], X[val_idx]
    train_y_old, val_y_old = y[train_idx], y[val_idx]

    train_X, train_y_old = augment_minority_class(train_X, train_y_old)
    train_y = np.array([[1,0] if l == 0 else [0,1] for l in train_y_old])
    val_y = np.array([[1,0] if l == 0 else [0,1] for l in val_y_old])

    # To flow train and val data into model
    train_generator = train_datagen.flow(
        x=train_X,
        y=train_y,
        batch_size=batch_size)

    validation_generator = validation_datagen.flow(
        x=val_X,
        y=val_y,
        shuffle=False,
        batch_size=batch_size)

    model.compile(optimizer=Adam(learning_rate=0.0001),
            loss="categorical_crossentropy",
            metrics=["binary_accuracy", "AUC"])

    epoch_history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        verbose=1)

    # Tracking metrics for base model training
    fold_trainloss = epoch_history.history["loss"]
    fold_trainacc = epoch_history.history["binary_accuracy"]
    fold_valloss = epoch_history.history["val_loss"]
    fold_valacc = epoch_history.history["val_binary_accuracy"]

    # Saving model for use later
    model.save_weights(fold_filepath)


    # Finetuning loop
    for layer_num in layers_frozen:

        # New file path for finetune model
        layer_filepath = f"./tf_exp/vggbasemodel_{fold_idx}_finetune_{layer_num}.h5"

        # Copy base model metrics to continue plot
        finetune_trainloss = fold_trainloss.copy()
        finetune_trainacc = fold_trainacc.copy()
        finetune_valloss = fold_valloss.copy()
        finetune_valacc = fold_valacc.copy()

        # Load base model for certain fold
        model = create_vgg16tf()
        model.load_weights(fold_filepath)

        # Freezing weights
        for id, layer in enumerate(model.layers):
            layer.trainable = False
            if id > layer_num:
                layer.trainable = True

        model.compile(optimizer=Adam(learning_rate=1e-4),
            loss="categorical_crossentropy",
            metrics=["binary_accuracy", "AUC"])

        epoch_history_ft = model.fit(
            train_generator,
            epochs=epochs_finetune,
            validation_data=validation_generator,
            verbose=1)
        model.save_weights(layer_filepath)

        # Continue plot with finetune metrics
        finetune_trainloss.extend(epoch_history_ft.history["loss"])
        finetune_trainacc.extend(epoch_history_ft.history["binary_accuracy"])
        finetune_valloss.extend(epoch_history_ft.history["val_loss"])
        finetune_valacc.extend(epoch_history_ft.history["val_binary_accuracy"])

        #Pickle dump training history
        finetune_history = {
            "loss": finetune_trainloss,
            "acc": finetune_trainacc,
            "val_loss": finetune_valloss,
            "val_acc": finetune_valacc
        }

        # Dumping finetune_history for sanity measures
        pickle.dump(finetune_history, open(f"./tf_exp/fold_{fold_idx}_finetune_{layer_num}.pkl", "wb"))

        loss, accuracy, auc = model.evaluate(validation_generator)
        preds = model.predict(validation_generator)
        buffer = [loss]
        buffer.extend(added_metrics(preds, val_y))

        layer_metrics[layer_num].append(buffer)

        print(f"Fold{fold_idx} Layer{layer_num}: {added_metrics(preds, val_y)}")

        # History.history plots
        fig, ax = plt.subplots(1,2,figsize=(13,6))

        ax[0].plot(finetune_trainacc, label="train_acc")
        ax[0].plot(finetune_valacc, label="val_acc")
        ax[0].axvline(x=epochs, ymin =0, ymax=10, color="red", alpha=0.5, linestyle="--")
        ax[0].set_xlabel("Epoch")
        ax[0].set_title(f"Fold {fold_idx+1}")
        ax[0].legend()

        ax[1].plot(finetune_trainloss, label="train_loss")
        ax[1].plot(finetune_valloss, label="val_loss")
        ax[1].axvline(x=epochs, ymin =0, ymax=10, color="red", alpha=0.5, linestyle="--")
        ax[1].set_xlabel("Epoch")
        ax[1].legend()
        plt.show()

        keras.backend.clear_session()

#### Transfer learning exp summary

##### Val predictions

In [ ]:
layers_frozen = [2, 3, 8, 9, 10, 12, 13, 14, 16, 17]
layers_frozen = [i + 2 for i in layers_frozen]              # Need to add 2 due to calculations (index start from 0 and id>layer_num)

layer_means = []
layer_errors = []
for layer_num, fold_metrics in layer_metrics.items():
    fold_metrics = np.array(fold_metrics)
    fold_means = np.mean(fold_metrics, axis = 0)
    fold_errors = np.std(fold_metrics, axis = 0)

    layer_means.append(fold_means)
    layer_errors.append(fold_errors)

means = list(map(list,(zip(*layer_means))))
errors = list(map(list,(zip(*layer_errors))))

print("\nMeans\n")
print(means)
print("\nErrors\n")
print(errors)
print("\n")

fig, ax = plt.subplots(1,3,figsize=(13,6))

ax[0].errorbar(layers_frozen, means[0], yerr=errors[0], label="val_loss")
ax[0].set_xlabel("Layers frozen")
ax[0].legend()

ax[1].errorbar(layers_frozen, means[1], yerr=errors[1], label="TN")
ax[1].errorbar(layers_frozen, means[2], yerr=errors[2], label="TP")
ax[1].errorbar(layers_frozen, means[3], yerr=errors[3], label="FN")
ax[1].errorbar(layers_frozen, means[4], yerr=errors[4], label="FP")
ax[1].set_xlabel("Layers frozen")
ax[1].legend()

ax[2].errorbar(layers_frozen, means[5], yerr=errors[5], label="precision")
ax[2].errorbar(layers_frozen, means[6], yerr=errors[6], label="accuracy")
ax[2].errorbar(layers_frozen, means[7], yerr=errors[7], label="recall")
ax[2].set_xlabel("Layers frozen")
ax[2].legend()

plt.show()

In [ ]:
# Copied the means and SD as the code block for the experiment took 2-3hrs to run
# Just in case it went missing, and to facilitate analysis after the session

layers_frozen = [2, 3, 8, 9, 10, 12, 13, 14, 16, 17]
layers_frozen = [i + 2 for i in layers_frozen]              # Need to add 2 due to calculations (index start from 0 and id>layer_num)

means = [[0.6460669994354248, 0.591467696428299, 0.6758981585502625, 0.5428489506244659, 0.6624845623970032, 0.4607712388038635, 0.4052444934844971, 0.5576117932796478, 0.4481421530246735, 0.39641469717025757], [6.4, 4.8, 4.8, 5.2, 8.8, 6.4, 8.2, 7.4, 6.6, 8.4], [12.2, 16.8, 14.6, 17.2, 10.2, 18.4, 18.6, 18.4, 19.0, 17.6], [4.0, 5.6, 5.6, 5.2, 1.6, 4.0, 2.2, 3.0, 3.8, 2.0], [8.0, 3.4, 5.6, 3.0, 10.0, 1.8, 1.6, 1.8, 1.2, 2.6], [0.6038095238095238, 0.8357142857142857, 0.7219047619047618, 0.8538095238095238, 0.5023809523809524, 0.9104761904761904, 0.9204761904761904, 0.9114285714285714, 0.9404761904761905, 0.8709523809523809], [0.6086021505376344, 0.7075268817204301, 0.6337634408602151, 0.7335483870967743, 0.6206451612903225, 0.8111827956989247, 0.8754838709677418, 0.843010752688172, 0.8369892473118281, 0.8494623655913978], [0.7700314889788574, 0.7627224627224628, 0.7711167182267795, 0.7853002070393376, 0.7214141414141414, 0.824503105590062, 0.8981254189492176, 0.8631032125768968, 0.8405035865905433, 0.9051171406777815]]
errors = [[0.05789509557340848, 0.14546233125374053, 0.09779906780071315, 0.14980663076625234, 0.15593145096142064, 0.18000572987292435, 0.1252150276740922, 0.19599881757688642, 0.12262273448221743, 0.04791983317594967], [2.4166091947189146, 2.4819347291981715, 3.249615361854384, 2.6381811916545836, 1.1661903789690602, 1.4966629547095764, 1.3266499161421599, 1.3564659966250536, 1.8547236990991407, 1.3564659966250536], [4.707440918375928, 4.069397989875161, 4.673328578219169, 2.481934729198171, 6.4, 1.2000000000000002, 1.0198039027185568, 1.019803902718557, 0.6324555320336759, 1.4966629547095764], [2.449489742783178, 2.939387691339814, 3.6110940170535577, 3.059411708155671, 1.3564659966250536, 1.6733200530681511, 1.16619037896906, 1.0954451150103321, 2.2271057451320084, 1.2649110640673518], [4.69041575982343, 4.454211490264017, 4.586937976471886, 2.8284271247461903, 6.2289646009589745, 0.9797958971132713, 0.8, 1.16619037896906, 0.4, 1.3564659966250536], [0.2346445181260822, 0.2116986691753452, 0.23004189700230093, 0.13473413814568083, 0.3133897763607448, 0.04938625585700636, 0.04036679895718616, 0.056892307862113946, 0.02025909274874535, 0.06826983942457085], [0.08846267338933866, 0.12368115684377644, 0.05821406468250258, 0.10061298998733334, 0.17192791849239608, 0.07355530370546157, 0.026130580953262834, 0.014183769847605276, 0.06687307623148353, 0.017668469596940826], [0.05549313187578253, 0.0789901210078336, 0.12456858000894624, 0.10278538753270194, 0.36533631826904783, 0.06429224519490624, 0.04705231632623448, 0.035720820157888085, 0.0744328107363998, 0.05546636624229973]]


fig, ax = plt.subplots(1,2,figsize=(13,6))

ax[0].plot(layers_frozen, means[0], label="val_loss", color="midnightblue")
ax[0].axvline(x = 11, ymin = 0, ymax=30, color="midnightblue", alpha = 0.3, linestyle="--")
ax[0].axvline(x = 15, ymin = 0, ymax=30, color="midnightblue", alpha = 0.3, linestyle="--")
ax[0].set_xlabel("Layers frozen")
ax[0].set_ylabel("Validation loss")
ax[0].legend()

ax[1].plot(layers_frozen, means[5], label="precision", color="midnightblue")
ax[1].plot(layers_frozen, means[6], label="accuracy", color="salmon")
ax[1].plot(layers_frozen, means[7], label="recall", color="lightpink")
ax[1].axvline(x = 11, ymin = 0, ymax=30, color="midnightblue", alpha = 0.3, linestyle="--")
ax[1].axvline(x = 15, ymin = 0, ymax=30, color="midnightblue", alpha = 0.3, linestyle="--")
ax[1].set_xlabel("Layers frozen")
ax[1].set_ylabel("Metrics score")
ax[1].legend()

plt.show()

In [ ]:
val_tfexp_means = [[0.6460669994354248, 0.591467696428299, 0.6758981585502625, 0.5428489506244659, 0.6624845623970032, 0.4607712388038635, 0.4052444934844971, 0.5576117932796478, 0.4481421530246735, 0.39641469717025757], [6.4, 4.8, 4.8, 5.2, 8.8, 6.4, 8.2, 7.4, 6.6, 8.4], [12.2, 16.8, 14.6, 17.2, 10.2, 18.4, 18.6, 18.4, 19.0, 17.6], [4.0, 5.6, 5.6, 5.2, 1.6, 4.0, 2.2, 3.0, 3.8, 2.0], [8.0, 3.4, 5.6, 3.0, 10.0, 1.8, 1.6, 1.8, 1.2, 2.6], [0.6038095238095238, 0.8357142857142857, 0.7219047619047618, 0.8538095238095238, 0.5023809523809524, 0.9104761904761904, 0.9204761904761904, 0.9114285714285714, 0.9404761904761905, 0.8709523809523809], [0.6086021505376344, 0.7075268817204301, 0.6337634408602151, 0.7335483870967743, 0.6206451612903225, 0.8111827956989247, 0.8754838709677418, 0.843010752688172, 0.8369892473118281, 0.8494623655913978], [0.7700314889788574, 0.7627224627224628, 0.7711167182267795, 0.7853002070393376, 0.7214141414141414, 0.824503105590062, 0.8981254189492176, 0.8631032125768968, 0.8405035865905433, 0.9051171406777815]]
val_tfexp_errors = [[0.05789509557340848, 0.14546233125374053, 0.09779906780071315, 0.14980663076625234, 0.15593145096142064, 0.18000572987292435, 0.1252150276740922, 0.19599881757688642, 0.12262273448221743, 0.04791983317594967], [2.4166091947189146, 2.4819347291981715, 3.249615361854384, 2.6381811916545836, 1.1661903789690602, 1.4966629547095764, 1.3266499161421599, 1.3564659966250536, 1.8547236990991407, 1.3564659966250536], [4.707440918375928, 4.069397989875161, 4.673328578219169, 2.481934729198171, 6.4, 1.2000000000000002, 1.0198039027185568, 1.019803902718557, 0.6324555320336759, 1.4966629547095764], [2.449489742783178, 2.939387691339814, 3.6110940170535577, 3.059411708155671, 1.3564659966250536, 1.6733200530681511, 1.16619037896906, 1.0954451150103321, 2.2271057451320084, 1.2649110640673518], [4.69041575982343, 4.454211490264017, 4.586937976471886, 2.8284271247461903, 6.2289646009589745, 0.9797958971132713, 0.8, 1.16619037896906, 0.4, 1.3564659966250536], [0.2346445181260822, 0.2116986691753452, 0.23004189700230093, 0.13473413814568083, 0.3133897763607448, 0.04938625585700636, 0.04036679895718616, 0.056892307862113946, 0.02025909274874535, 0.06826983942457085], [0.08846267338933866, 0.12368115684377644, 0.05821406468250258, 0.10061298998733334, 0.17192791849239608, 0.07355530370546157, 0.026130580953262834, 0.014183769847605276, 0.06687307623148353, 0.017668469596940826], [0.05549313187578253, 0.0789901210078336, 0.12456858000894624, 0.10278538753270194, 0.36533631826904783, 0.06429224519490624, 0.04705231632623448, 0.035720820157888085, 0.0744328107363998, 0.05546636624229973]]

layers_frozen = [2, 3, 8, 9, 10, 12, 13, 14, 16, 17]
layers_frozen = [i + 2 for i in layers_frozen]              # Need to add 2 due to calculations (index start from 0 and id>layer_num)

val_tf_exp = pd.DataFrame(np.array(val_tfexp_means).T.tolist(), columns = ["Val_loss", "TN", "TP", "FN", "FP", "Precision", "Accuracy", "Recall"])
val_tf_exp["Layers frozen"] = layers_frozen
val_tf_exp = val_tf_exp.set_index("Layers frozen")
val_tf_exp


##### Test predictions

In [ ]:
layers_frozen = [2, 3, 8, 9, 10, 12, 13, 14, 16, 17]
batch_size = 64

test_X_new = np.array(test_X)
test_y_new = np.array([[1,0] if l == 0 else [0,1] for l in test_y])

test_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True)

test_generator = test_datagen.flow(
    x=test_X_new,
    y=test_y_new,
    shuffle=False,
    batch_size=batch_size)

layer_ft_mean = []
layer_ft_error = []

for layer_num in layers_frozen:
    fold_buffer = []
    for fold_idx in range(5):
        model = create_vgg16tf()
        model.load_weights(f"./tf_exp/vggbasemodel_{fold_idx}_finetune_{layer_num}.h5")

        model.compile(optimizer=Adam(learning_rate=1e-4),
            loss="categorical_crossentropy",
            metrics=["binary_accuracy", "AUC"])

        preds = model.predict(test_generator)
        fold_buffer.append(added_metrics(preds, test_y_new))
    layer_ft_mean.append(np.mean(fold_buffer, axis = 0))
    layer_ft_error.append(np.std(fold_buffer, axis = 0))

means = list(map(list,(zip(*layer_ft_mean))))
errors = list(map(list,(zip(*layer_ft_error))))

print("\nMeans\n")
print(means)
print("\nErrors\n")
print(errors)
print("\n")

In [ ]:
means = [[6.2, 4.0, 4.4, 5.4, 7.4, 4.8, 5.4, 5.8, 5.0, 5.4], [16.8, 25.8, 23.0, 22.6, 15.4, 24.6, 24.8, 26.2, 24.4, 22.6], [3.8, 6.0, 5.6, 4.6, 2.6, 5.2, 4.6, 4.2, 5.0, 4.6], [12.2, 3.2, 6.0, 6.4, 13.6, 4.4, 4.2, 2.8, 4.6, 6.4], [0.5793103448275863, 0.8896551724137932, 0.793103448275862, 0.7793103448275862, 0.5310344827586206, 0.8482758620689657, 0.8551724137931034, 0.903448275862069, 0.8413793103448276, 0.7793103448275861], [0.5897435897435898, 0.764102564102564, 0.7025641025641025, 0.7179487179487178, 0.5846153846153846, 0.7538461538461539, 0.7743589743589743, 0.8205128205128206, 0.7538461538461538, 0.7179487179487178], [0.8086697722567289, 0.8134637460040686, 0.8316794091641961, 0.8339805351848962, 0.8360027472527471, 0.8231996348931834, 0.8437950999686261, 0.8621506734006734, 0.8292578541118585, 0.8312282149523528]]
errors = [[1.6, 1.4142135623730951, 3.2619012860600183, 1.4966629547095764, 2.0591260281974, 0.7483314773547882, 0.48989794855663565, 0.7483314773547882, 0.0, 0.48989794855663565], [7.626270385975047, 3.1874754901018454, 5.89915248150105, 4.498888751680797, 8.138795979750322, 3.0724582991474434, 1.4696938456699067, 1.7204650534085253, 1.854723699099141, 1.2], [1.6, 1.4142135623730951, 3.2619012860600183, 1.4966629547095764, 2.0591260281974, 0.7483314773547882, 0.48989794855663565, 0.7483314773547882, 0.0, 0.48989794855663565], [7.626270385975047, 3.1874754901018454, 5.89915248150105, 4.498888751680797, 8.138795979750322, 3.0724582991474434, 1.4696938456699071, 1.7204650534085255, 1.8547236990991407, 1.2], [0.2629748408956913, 0.1099129479345464, 0.2034190510862431, 0.1551340948855447, 0.2806481372327697, 0.10594683790163598, 0.05067909812654853, 0.05932638115201811, 0.0639559896241083, 0.041379310344827586], [0.15554616295490362, 0.054754247446314415, 0.07360359022772989, 0.0959399329942036, 0.17571119691294165, 0.08970695222838923, 0.029902317409463055, 0.042905642386362845, 0.047557017925618984, 0.0229340305384594], [0.030103984222125443, 0.022015450511969507, 0.08832593675252555, 0.03969442682711525, 0.13566344923656123, 0.03469455269134492, 0.010006692278627872, 0.020417497382018646, 0.010696286385120024, 0.010914639976811802]]

layers_frozen = [2, 3, 8, 9, 10, 12, 13, 14, 16, 17]
layers_frozen = [i + 2 for i in layers_frozen]              # Need to add 2 due to calculations (index start from 0 and id>layer_num)

fig, ax = plt.subplots(1,2,figsize=(13,6))

ax[0].errorbar(layers_frozen, means[0], label="TN")
ax[0].errorbar(layers_frozen, means[1], label="TP")
ax[0].errorbar(layers_frozen, means[2], label="FN")
ax[0].errorbar(layers_frozen, means[3], label="FP")
ax[0].axvline(x = 12, ymin = 0, ymax=30, color="red", alpha = 0.3, linestyle="--")
ax[0].axvline(x = 16, ymin = 0, ymax=30, color="red", alpha = 0.3, linestyle="--")
ax[0].axvline(x = 11, ymin = 0, ymax=30, color="green", alpha = 0.3, linestyle="--")
ax[0].axvline(x = 15, ymin = 0, ymax=30, color="green", alpha = 0.3, linestyle="--")
ax[0].axvline(x = 18, ymin = 0, ymax=30, color="green", alpha = 0.3, linestyle="--")
ax[0].set_xlabel("Layers frozen")
ax[0].legend()

ax[1].errorbar(layers_frozen, means[4], label="precision")
ax[1].errorbar(layers_frozen, means[5], label="accuracy")
ax[1].errorbar(layers_frozen, means[6], label="recall")
ax[1].axvline(x = 12, ymin = 0, ymax=30, color="red", alpha = 0.3, linestyle="--")
ax[1].axvline(x = 16, ymin = 0, ymax=30, color="red", alpha = 0.3, linestyle="--")
ax[1].axvline(x = 11, ymin = 0, ymax=30, color="green", alpha = 0.3, linestyle="--")
ax[1].axvline(x = 15, ymin = 0, ymax=30, color="green", alpha = 0.3, linestyle="--")
ax[1].axvline(x = 18, ymin = 0, ymax=30, color="green", alpha = 0.3, linestyle="--")
ax[1].set_xlabel("Layers frozen")
ax[1].legend()

plt.show()

In [ ]:
layers_frozen = [2, 3, 8, 9, 10, 12, 13, 14, 16, 17]
layers_frozen = [i + 2 for i in layers_frozen]              # Need to add 2 due to calculations (index start from 0 and id>layer_num)

test_tfexp_means = [[6.2, 4.0, 4.4, 5.4, 7.4, 4.8, 5.4, 5.8, 5.0, 5.4], [16.8, 25.8, 23.0, 22.6, 15.4, 24.6, 24.8, 26.2, 24.4, 22.6], [3.8, 6.0, 5.6, 4.6, 2.6, 5.2, 4.6, 4.2, 5.0, 4.6], [12.2, 3.2, 6.0, 6.4, 13.6, 4.4, 4.2, 2.8, 4.6, 6.4], [0.5793103448275863, 0.8896551724137932, 0.793103448275862, 0.7793103448275862, 0.5310344827586206, 0.8482758620689657, 0.8551724137931034, 0.903448275862069, 0.8413793103448276, 0.7793103448275861], [0.5897435897435898, 0.764102564102564, 0.7025641025641025, 0.7179487179487178, 0.5846153846153846, 0.7538461538461539, 0.7743589743589743, 0.8205128205128206, 0.7538461538461538, 0.7179487179487178], [0.8086697722567289, 0.8134637460040686, 0.8316794091641961, 0.8339805351848962, 0.8360027472527471, 0.8231996348931834, 0.8437950999686261, 0.8621506734006734, 0.8292578541118585, 0.8312282149523528]]
test_tfexp_errors = [[1.6, 1.4142135623730951, 3.2619012860600183, 1.4966629547095764, 2.0591260281974, 0.7483314773547882, 0.48989794855663565, 0.7483314773547882, 0.0, 0.48989794855663565], [7.626270385975047, 3.1874754901018454, 5.89915248150105, 4.498888751680797, 8.138795979750322, 3.0724582991474434, 1.4696938456699067, 1.7204650534085253, 1.854723699099141, 1.2], [1.6, 1.4142135623730951, 3.2619012860600183, 1.4966629547095764, 2.0591260281974, 0.7483314773547882, 0.48989794855663565, 0.7483314773547882, 0.0, 0.48989794855663565], [7.626270385975047, 3.1874754901018454, 5.89915248150105, 4.498888751680797, 8.138795979750322, 3.0724582991474434, 1.4696938456699071, 1.7204650534085255, 1.8547236990991407, 1.2], [0.2629748408956913, 0.1099129479345464, 0.2034190510862431, 0.1551340948855447, 0.2806481372327697, 0.10594683790163598, 0.05067909812654853, 0.05932638115201811, 0.0639559896241083, 0.041379310344827586], [0.15554616295490362, 0.054754247446314415, 0.07360359022772989, 0.0959399329942036, 0.17571119691294165, 0.08970695222838923, 0.029902317409463055, 0.042905642386362845, 0.047557017925618984, 0.0229340305384594], [0.030103984222125443, 0.022015450511969507, 0.08832593675252555, 0.03969442682711525, 0.13566344923656123, 0.03469455269134492, 0.010006692278627872, 0.020417497382018646, 0.010696286385120024, 0.010914639976811802]]

test_tf_exp = pd.DataFrame(np.array(test_tfexp_means).T.tolist(), columns = ["TN", "TP", "FN", "FP", "Precision", "Accuracy", "Recall"])
test_tf_exp["Layers frozen"] = layers_frozen
test_tf_exp = test_tf_exp.set_index("Layers frozen")
test_tf_exp


## <span style = "font-family: Cambria">Final models</span>

#### <span style = "font-family: Cambria">VGG16TF - All frozen</span>

##### Run

In [ ]:
batch_size = 64
epochs = 15

vgg16tf = create_vgg16tf()

# Augmenting train data
X_aug, y_aug = augment_minority_class(X, y)
y_aug_new = np.array([[1,0] if l == 0 else [0,1] for l in y_aug])

train_generator = train_datagen.flow(
    x=X_aug,
    y=y_aug_new,
    batch_size=batch_size)

# Test data
test_X_new = np.array(test_X)
test_y_new = np.array([[1,0] if l == 0 else [0,1] for l in test_y])

test_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True)

test_generator = test_datagen.flow(
    x=test_X_new,
    y=test_y_new,
    shuffle=False,
    batch_size=batch_size)

optimizer = Adam(learning_rate=0.00005)

vgg16tf.compile(optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=["binary_accuracy", "AUC"])

early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True, start_from_epoch=3)

epoch_history = vgg16tf.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[early_stopping],
    verbose=1)


vggtf_trainloss = epoch_history.history["loss"].copy()
vggtf_trainauc = epoch_history.history["auc"].copy()
vggtf_valloss = epoch_history.history["val_loss"].copy()
vggtf_valauc = epoch_history.history["val_auc"].copy()


# Finetuned results
loss, accuracy, auc = vgg16tf.evaluate(test_generator)
preds = vgg16tf.predict(test_generator)
print(added_metrics(preds, test_y_new))

# Saving weights of finetune model
vgg16tf.save_weights(f"./tfexp/VGG16TF_ft.h5")


# History.history plots
fig, ax = plt.subplots(1,2,figsize=(13,6))

ax[0].plot(vggtf_trainauc, label="train_auc", color = "midnightblue")
ax[0].plot(vggtf_valauc, label="test_auc", color = "salmon")
ax[0].set_xlabel("Epoch")
ax[0].legend()

ax[1].plot(vggtf_trainloss, label="train_loss", color = "midnightblue")
ax[1].plot(vggtf_valloss, label="test_loss", color = "salmon")
ax[1].set_xlabel("Epoch")
ax[1].legend()
plt.show()

##### Gradcam

In [ ]:
grad_heatmap(vgg16tf, X_aug, y_aug_new, "block5_conv3", dim=(7,10))

In [ ]:
grad_heatmap(vgg16tf, test_X, test_y_new, "block5_conv3", dim=(4,10))

#### <span style = "font-family: Cambria">VGG16TF_11 - Training layer 11 onwards</span>

##### Run

In [ ]:
base_filepath = f"./tf_exp_6apr/vgg16tf_11_base.h5"
ft_filepath = f"./tf_exp_6apr/vgg16tf_11_ft.h5"
layer_num = 9

batch_size = 64
epochs = 15
epochs_finetune = 7


vgg16tf_11 = create_vgg16tf()

# Augmenting train data
X_aug, y_aug = augment_minority_class(X, y)
y_aug_new = np.array([[1,0] if l == 0 else [0,1] for l in y_aug])

train_generator = train_datagen.flow(
    x=X_aug,
    y=y_aug_new,
    batch_size=batch_size)

# Test data
test_X_new = np.array(test_X)
test_y_new = np.array([[1,0] if l == 0 else [0,1] for l in test_y])

test_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True)

test_generator = test_datagen.flow(
    x=test_X_new,
    y=test_y_new,
    shuffle=False,
    batch_size=batch_size)

optimizer = Adam(learning_rate=0.0005)

# Initial learning rate
vgg16tf_11.compile(optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=["binary_accuracy", "AUC"])

early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True, start_from_epoch=0)

epoch_history = vgg16tf_11.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[early_stopping],
    verbose=1)

# Interim result
loss, accuracy, auc = vgg16tf_11.evaluate(test_generator)
preds = vgg16tf_11.predict(test_generator)
print(added_metrics(preds, test_y_new))

# Saving weights of base model
vgg16tf_11.save_weights(base_filepath)

vggtf_11_trainloss = epoch_history.history["loss"].copy()
vggtf_11_trainauc = epoch_history.history["auc"].copy()
vggtf_11_valloss = epoch_history.history["val_loss"].copy()
vggtf_11_valauc = epoch_history.history["val_auc"].copy()
n = len(epoch_history.history["loss"])

# Unfrozen layers 11 onwards for finetuning
for id, layer in enumerate(vgg16tf_11.layers):
    layer.trainable = False
    if id > layer_num:
        layer.trainable = True

optimizer.learning_rate.assign(1e-5)

epoch_history_ft = vgg16tf_11.fit(
        train_generator,
        epochs=epochs_finetune,
        validation_data=test_generator,
        callbacks=[early_stopping],
        verbose=1)

vggtf_11_trainloss.extend(epoch_history_ft.history["loss"].copy())
vggtf_11_trainauc.extend(epoch_history_ft.history["auc"].copy())
vggtf_11_valloss.extend(epoch_history_ft.history["val_loss"].copy())
vggtf_11_valauc.extend(epoch_history_ft.history["val_auc"].copy())

# Finetuned results
loss, accuracy, auc = vgg16tf_11.evaluate(test_generator)
preds = vgg16tf_11.predict(test_generator)
print(added_metrics(preds, test_y_new))

# Saving weights of finetune model
vgg16tf_11.save_weights(f"./tfexp/vgg16tf_11_ft.h5")

# History.history plots
fig, ax = plt.subplots(1,2,figsize=(13,6))

ax[0].plot(vggtf_11_trainauc, label="train_auc", color="midnightblue")
ax[0].plot(vggtf_11_valauc, label="test_auc", color="salmon")
ax[0].axvline(x=n, ymin=0, ymax=10, color="midnightblue", alpha=0.5, linestyle="--")
ax[0].set_xlabel("Epoch")
ax[0].legend()

ax[1].plot(vggtf_11_trainloss, label="train_loss", color="midnightblue")
ax[1].plot(vggtf_11_valloss, label="test_loss", color="salmon")
ax[1].axvline(x=n, ymin=0, ymax=10, color="midnightblue", alpha=0.5, linestyle="--")
ax[1].set_xlabel("Epoch")
ax[1].legend()
plt.show()


##### Gradcam

In [ ]:
grad_heatmap(vgg16tf_11, X_aug, y_aug_new, "block5_conv3", dim=(7,10))

In [ ]:
grad_heatmap(vgg16tf_11, test_X, test_y_new, "block5_conv3", dim=(4,10))

#### <span style = "font-family: Cambria">VGG16TF_15 - Training layer 15 onwards</span>

##### Run

In [ ]:
base_filepath = f"./tf_exp_6apr/vgg16tf_15_base.h5"
ft_filepath = f"./tf_exp_6apr/vgg16tf_15_ft.h5"
layer_num = 13

batch_size = 64
epochs = 15
epochs_finetune = 7


vgg16tf_15 = create_vgg16tf()

# Augmenting train data
X_aug, y_aug = augment_minority_class(X, y)
y_aug_new = np.array([[1,0] if l == 0 else [0,1] for l in y_aug])

train_generator = train_datagen.flow(
    x=X_aug,
    y=y_aug_new,
    batch_size=batch_size)

# Test data
test_X_new = np.array(test_X)
test_y_new = np.array([[1,0] if l == 0 else [0,1] for l in test_y])

test_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True)

test_generator = test_datagen.flow(
    x=test_X_new,
    y=test_y_new,
    shuffle=False,
    batch_size=batch_size)

optimizer = Adam(learning_rate=0.0005)

# Initial learning rate
vgg16tf_15.compile(optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=["binary_accuracy", "AUC"])

early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True, start_from_epoch=0)

epoch_history = vgg16tf_15.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[early_stopping],
    verbose=1)

# Interim result
loss, accuracy, auc = vgg16tf_15.evaluate(test_generator)
preds = vgg16tf_15.predict(test_generator)
print(added_metrics(preds, test_y_new))

# Saving weights of base model
vgg16tf_15.save_weights(base_filepath)

vggtf_15_trainloss = epoch_history.history["loss"].copy()
vggtf_15_trainauc = epoch_history.history["auc"].copy()
vggtf_15_valloss = epoch_history.history["val_loss"].copy()
vggtf_15_valauc = epoch_history.history["val_auc"].copy()
n = len(epoch_history.history["loss"])

# Unfrozen layers 15 onwards for finetuning
for id, layer in enumerate(vgg16tf_15.layers):
    layer.trainable = False
    if id > layer_num:
        layer.trainable = True

optimizer.learning_rate.assign(1e-5)

epoch_history_ft = vgg16tf_15.fit(
        train_generator,
        epochs=epochs_finetune,
        validation_data=test_generator,
        callbacks=[early_stopping],
        verbose=1)

vggtf_15_trainloss.extend(epoch_history_ft.history["loss"].copy())
vggtf_15_trainauc.extend(epoch_history_ft.history["auc"].copy())
vggtf_15_valloss.extend(epoch_history_ft.history["val_loss"].copy())
vggtf_15_valauc.extend(epoch_history_ft.history["val_auc"].copy())

# Finetuned results
loss, accuracy, auc = vgg16tf_15.evaluate(test_generator)
preds = vgg16tf_15.predict(test_generator)
print(added_metrics(preds, test_y_new))

# Saving weights of finetune model
vgg16tf_15.save_weights(f"./tfexp/vgg16tf_15_ft.h5")

# History.history plots
fig, ax = plt.subplots(1,2,figsize=(13,6))

ax[0].plot(vggtf_15_trainauc, label="train_auc", color="midnightblue")
ax[0].plot(vggtf_15_valauc, label="test_auc", color="salmon")
ax[0].axvline(x=n, ymin=0, ymax=10, color="midnightblue", alpha=0.5, linestyle="--")
ax[0].set_xlabel("Epoch")
ax[0].legend()

ax[1].plot(vggtf_15_trainloss, label="train_loss", color="midnightblue")
ax[1].plot(vggtf_15_valloss, label="test_loss", color="salmon")
ax[1].axvline(x=n, ymin=0, ymax=10, color="midnightblue", alpha=0.5, linestyle="--")
ax[1].set_xlabel("Epoch")
ax[1].legend()
plt.show()


##### Gradcam

In [ ]:
grad_heatmap(vgg16tf_15, X_aug, y_aug_new, "block5_conv3", dim=(7,10))

In [ ]:
grad_heatmap(vgg16tf_15, test_X, test_y_new, "block5_conv3", dim=(4,10))

#### <span style = "font-family: Cambria">VGG16TF_18 - Training layer 18</span>

##### Run

In [ ]:
base_filepath = f"./tf_exp_6apr/VGG16TF_18_base.h5"
ft_filepath = f"./tf_exp_6apr/VGG16TF_18_ft.h5"
layer_num = 16

batch_size = 64
epochs = 15
epochs_finetune = 7

vgg16tf_18 = create_vgg16tf()

# Augmenting train data
X_aug, y_aug = augment_minority_class(X, y)
y_aug_new = np.array([[1,0] if l == 0 else [0,1] for l in y_aug])

train_generator = train_datagen.flow(
    x=X_aug,
    y=y_aug_new,
    batch_size=batch_size)

# Test data
test_X_new = np.array(test_X)
test_y_new = np.array([[1,0] if l == 0 else [0,1] for l in test_y])

test_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True)

test_generator = test_datagen.flow(
    x=test_X_new,
    y=test_y_new,
    shuffle=False,
    batch_size=batch_size)

optimizer = Adam(learning_rate=0.0005)

# Initial learning rate
vgg16tf_18.compile(optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=["binary_accuracy", "AUC"])

early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True, start_from_epoch=0)

epoch_history = vgg16tf_18.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[early_stopping],
    verbose=1)

# Interim result
loss, accuracy, auc = vgg16tf_18.evaluate(test_generator)
preds = vgg16tf_18.predict(test_generator)
print(added_metrics(preds, test_y_new))

# Saving weights of base model
vgg16tf_18.save_weights(base_filepath)

vggtf_18_trainloss = epoch_history.history["loss"].copy()
vggtf_18_trainauc = epoch_history.history["auc"].copy()
vggtf_18_valloss = epoch_history.history["val_loss"].copy()
vggtf_18_valauc = epoch_history.history["val_auc"].copy()
n = len(epoch_history.history["loss"])

# Unfrozen layers 18 onwards for finetuning
for id, layer in enumerate(vgg16tf_18.layers):
    layer.trainable = False
    if id > layer_num:
        layer.trainable = True

optimizer.learning_rate.assign(1e-5)

epoch_history_ft = vgg16tf_18.fit(
        train_generator,
        epochs=epochs_finetune,
        validation_data=test_generator,
        callbacks=[early_stopping],
        verbose=1)

vggtf_18_trainloss.extend(epoch_history_ft.history["loss"].copy())
vggtf_18_trainauc.extend(epoch_history_ft.history["auc"].copy())
vggtf_18_valloss.extend(epoch_history_ft.history["val_loss"].copy())
vggtf_18_valauc.extend(epoch_history_ft.history["val_auc"].copy())

# Finetuned results
loss, accuracy, auc = vgg16tf_18.evaluate(test_generator)
preds = vgg16tf_18.predict(test_generator)

print(added_metrics(preds, test_y_new))

# Saving weights of finetune model
vgg16tf_18.save_weights(f"./tfexp/vgg16tf_18_ft.h5")

# History.history plots
fig, ax = plt.subplots(1,2,figsize=(13,6))

ax[0].plot(vggtf_18_trainauc, label="train_auc", color="midnightblue")
ax[0].plot(vggtf_18_valauc, label="test_auc", color="salmon")
ax[0].axvline(x=n, ymin=0, ymax=10, color="midnightblue", alpha=0.5, linestyle="--")
ax[0].set_xlabel("Epoch")
ax[0].legend()

ax[1].plot(vggtf_18_trainloss, label="train_loss", color="midnightblue")
ax[1].plot(vggtf_18_valloss, label="test_loss", color="salmon")
ax[1].axvline(x=n, ymin=0, ymax=10, color="midnightblue", alpha=0.5, linestyle="--")
ax[1].set_xlabel("Epoch")
ax[1].legend()
plt.show()


##### Gradcam

In [ ]:
grad_heatmap(vgg16tf_18, X_aug, y_aug_new, "block5_conv3", dim=(7,10))

In [ ]:
grad_heatmap(vgg16tf_18, test_X, test_y_new, "block5_conv3", dim=(4,10))

# <a id = "xgboost"></a><span style = "font-family: Cambria">**VGG16-XgBoost model**</span>

In [ ]:
# Data Splitting
benign_train_image, benign_test_image, benign_train_image_label, benign_test_image_label = train_test_split(b_imgs, b_label, test_size = 0.2)
malignant_train_image, malignant_test_image, malignant_train_image_label, malignant_test_image_label = train_test_split(m_imgs, m_label, test_size = 0.1)

train_image = np.append(benign_train_image, malignant_train_image, axis = 0)
test_image = np.append(benign_test_image, malignant_test_image, axis = 0)
train_label = np.append(benign_train_image_label, malignant_train_image_label, axis = 0)
test_label = np.append(benign_test_image_label, malignant_test_image_label, axis = 0)

In [ ]:
print(np.shape(train_image))
print(np.shape(test_image))

In [ ]:
# Data Augmentation
augmented_benign_train_image = augment_images_xgboost(benign_train_image, 600, seed = random.randint(1, 2024))
augmented_malignant_train_image = augment_images_xgboost(malignant_train_image, 600, seed = random.randint(1, 2024))

train_image = np.append(augmented_benign_train_image, augmented_malignant_train_image, axis = 0)

benign_labels = np.zeros((len(augmented_benign_train_image),))
malignant_labels = np.ones((len(augmented_malignant_train_image),))

train_label = np.append(benign_labels, malignant_labels, axis = 0)

In [ ]:
print(np.shape(augmented_benign_train_image))
print(np.shape(augmented_malignant_train_image))
print(np.shape(train_label))

In [ ]:
print(np.shape(train_image))
print(np.shape(test_image))
print(np.shape(train_label))
print(np.shape(test_label))

## <span style = "font-family: Cambria">CNN for Feature Extraction by CNN </span>

In [ ]:
feature_extractor_model = VGG16(weights = 'imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in feature_extractor_model.layers:
	layer.trainable = False

train_features = feature_extractor_model.predict(train_image)
reshaped_train_features = train_features.reshape(train_features.shape[0], -1)

test_features = feature_extractor_model.predict(test_image)
reshaped_test_features = test_features.reshape(test_features.shape[0], -1)

In [ ]:
print(np.shape(reshaped_train_features))
print(np.shape(reshaped_test_features))

## <span style = "font-family: Cambria">PCA Model </span>

We explored PCA to reduce the dimensions of flatten features from CNN while still retaining variability

In [ ]:
pca_model = PCA(n_components = 1000)
feature_pca = pca_model.fit_transform(reshaped_train_features)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(feature_pca[:, 0], feature_pca[:, 1], feature_pca[:, 2], c = train_label, cmap='viridis')
plt.colorbar(scatter, label='Digit', ticks=range(10))
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
ax.view_init(30, 75)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(range(1, len(pca_model.explained_variance_ratio_) + 1), pca_model.explained_variance_ratio_, alpha=0.5, align='center')
plt.step(range(1, len(pca_model.explained_variance_ratio_) + 1), np.cumsum(pca_model.explained_variance_ratio_), where='mid')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.title('Explained Variance vs. Number of Principal Components')
plt.axhline(y=0.8, color='r', linestyle='--', label='Threshold (0.8)') # Shown at approx. 250
plt.axhline(y=0.9, color='b', linestyle='--', label='Threshold (0.9)') # Shown at approx. 450
plt.show()

In [ ]:
model = SVC(kernel='linear')
clf = model.fit(reshaped_train_features, train_label)

z = lambda x,y: (clf.coef_[0][0]*x +clf.coef_[0][1]*y) / (clf.coef_[0][2]+0.1)

# Plot the decision plane in 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(feature_pca[:, 0], feature_pca[:, 2], feature_pca[:, 1], c = train_label, cmap='viridis')
plt.colorbar(scatter, label='Digit', ticks=range(10))
ax.set_xlabel('Principal Component 2')
ax.set_ylabel('Principal Component 3')
ax.set_zlabel('Principal Component 1')

tmp = np.linspace(-400, 400,10)
x,y = np.meshgrid(tmp,tmp)
# Plot the decision plane surface
ax.plot_surface(x, y, z(x,y))
ax.view_init(20, 45)
plt.show()

This shows how PCA could make a significant distinction across the images

In [ ]:
# Final PCA Model

pca_model = PCA(n_components = 450)
reduced_features_450 = pca_model.fit_transform(reshaped_train_features)
reduced_features_250 = reduced_features_450[:, :250]

In [ ]:
print(np.shape(reduced_features_450))
print(np.shape(reduced_features_250))

In [ ]:
test_features_450 = pca_model.transform(reshaped_test_features)
test_features_250 = test_features_450[:,:250]

In [ ]:
print(np.shape(test_features_450))
print(np.shape(test_features_250))

## <span style = "font-family: Cambria">XGBoost using the extracted features</span>


In [ ]:
def data_transform(features, labels):
  matrix = XGBoost.DMatrix(features, label = labels)
  return matrix


def model_fitting(model, features, labels):

    xgb_param = model.get_xgb_params()
    xgtrain = data_transform(features, labels = labels)
    cvresult = XGBoost.cv(xgb_param, xgtrain, num_boost_round=model.get_params()['n_estimators'], nfold = 5,
            metrics='auc', early_stopping_rounds = 20)
    model.set_params(n_estimators = cvresult.shape[0])

    #Fit the algorithm on the data
    model.fit(features, labels, eval_metric='auc')

    #Predict training set:
    dtrain_predictions = model.predict(features)
    dtrain_predprob = model.predict_proba(features)[:,1]

    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : ")
    print(sklearn.metrics.accuracy_score(labels, dtrain_predictions))
    print(" ")
    print ("AUC Score (Train): ")
    print(sklearn.metrics.roc_auc_score(labels, dtrain_predprob))

    feat_imp = pd.Series(model.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind = 'bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

### <span style = "font-family: Cambria">Initializing XGBoost</span>

In [ ]:
initial_XGBoost = XGBoost.XGBClassifier(learning_rate = 0.1,
                                n_estimators = 1000,
                                max_depth = 5,
                                min_child_weight = 1,
                                gamma = 0,
                                subsample = 0.8,
                                colsample_bytree = 0.8,
                                objective = 'binary:logistic',
                                nthread = 4,
                                scale_pos_weight = 1,
                                seed = 42)

### <span style = "font-family: Cambria">1. Searching for the best n_estimators</span>

In [ ]:
xgb_param = initial_XGBoost.get_xgb_params()
xgtrain = data_transform(reshaped_train_features, labels = train_label)
cvresult = XGBoost.cv(xgb_param, xgtrain, num_boost_round = initial_XGBoost.get_params()['n_estimators'], nfold = 5,
            metrics='auc', early_stopping_rounds = 20)
initial_XGBoost.set_params(n_estimators = cvresult.shape[0])

#Fit the algorithm on the data
initial_XGBoost.fit(reshaped_train_features, train_label, eval_metric='auc')

print(cvresult.shape[0])

The result of the best number of trees (n_estimators) is 449

### <span style = "font-family: Cambria">2. Tuning max_depth and min_child_weight </span>

In [ ]:
second_XGBoost = XGBoost.XGBClassifier(learning_rate = 0.1,
                                                  n_estimators = 449,
                                                  max_depth=5,
                                                  min_child_weight=1,
                                                  gamma=0,
                                                  subsample=0.8,
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic',
                                                  nthread=4,
                                                  scale_pos_weight=1,
                                                  seed = 42)

In [ ]:
param_grid_2 = {
 'max_depth':range(3,10,1),
 'min_child_weight':range(1,6,1)
}

In [ ]:
grid_search_2 = GridSearchCV(second_XGBoost, param_grid = param_grid_2, scoring = 'roc_auc', n_jobs = 1 , cv = 5, verbose = 2)

grid_search_2.fit(reshaped_train_features,train_label)
print(grid_search_2.best_estimator_)
print(grid_search_2.best_params_)
print(grid_search_2.best_score_)

The best parameters for both max_depth and min_child_weight is 3 and 1 respectively.

The best auc score for this classifier is 0.9921755829903978

### <span style = "font-family: Cambria">3. Tuning Gamma</span>

In [ ]:
third_XGBoost = XGBoost.XGBClassifier(learning_rate = 0.1,
                                                  n_estimators = 449,
                                                  max_depth=3,
                                                  min_child_weight=1,
                                                  gamma=0,
                                                  subsample=0.8,
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic',
                                                  nthread=4,
                                                  scale_pos_weight=1,
                                                  seed = 42)

In [ ]:
param_grid_3 = {
 'gamma':[i/10.0 for i in range(0,11)]
}

In [ ]:
grid_search_3 = GridSearchCV(third_XGBoost, param_grid = param_grid_3, scoring = 'roc_auc', n_jobs = 1 , cv = 5, verbose = 2)

grid_search_3.fit(reshaped_train_features,train_label)
print(grid_search_3.best_estimator_)
print(grid_search_3.best_params_)
print(grid_search_3.best_score_)

Best gamma is obtained when gamma = 0.1 with a total auc score of 0.9931851851851852

### <span style = "font-family: Cambria">4. Tuning subsample and colsample_by_tree</span>

In [ ]:
fourth_XGBoost = XGBoost.XGBClassifier(learning_rate = 0.1,
                                                  n_estimators = 449,
                                                  max_depth=3,
                                                  min_child_weight=1,
                                                  gamma=0.1,
                                                  subsample=0.8,
                                                  colsample_bytree=0.8,
                                                  objective= 'binary:logistic',
                                                  nthread=4,
                                                  scale_pos_weight=1,
                                                  seed = 42)

In [ ]:
param_grid_4 = {
 'subsample':[i/10.0 for i in range(5,11)],
 'colsample_bytree':[i/10.0 for i in range(5,11)]
}

In [ ]:
grid_search_4 = GridSearchCV(fourth_XGBoost, param_grid = param_grid_4, scoring = 'roc_auc', n_jobs = 5, cv = 5, verbose = 2)

grid_search_4.fit(reshaped_train_features,train_label)
print(grid_search_4.best_estimator_)
print(grid_search_4.best_params_)
print(grid_search_4.best_score_)

Best colsample_bytree and subsample is 0.6 and 0.6 respectively, with a final auc score of 0.9932181069958848

### <span style = "font-family: Cambria">5. Tuning the regularization method</span>

In [ ]:
fifth_XGBoost = XGBoost.XGBClassifier(learning_rate = 0.1,
                                                  n_estimators = 449,
                                                  max_depth=3,
                                                  min_child_weight=1,
                                                  gamma=0.1,
                                                  subsample=0.6,
                                                  colsample_bytree=0.6,
                                                  objective= 'binary:logistic',
                                                  nthread=4,
                                                  scale_pos_weight=1,
                                                  seed = 42)

In [ ]:
param_grid_5 = {
 'reg_alpha':[1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 10, 100]
}

In [ ]:
grid_search_5 = GridSearchCV(fifth_XGBoost, param_grid = param_grid_5, scoring = 'roc_auc', n_jobs = 1 , cv = 5, verbose = 2)

grid_search_5.fit(reshaped_train_features,train_label)
print(grid_search_5.best_estimator_)
print(grid_search_5.best_params_)
print(grid_search_5.best_score_)

print(grid_searh_5.best_params_["reg_alpha"])

Best reg_alpha is 0.1 with the final auc_score 0.9934156378600824

### <span style = "font-family: Cambria">6. Minimizing learning rate, and adjusting number of boosting rounds for better results</span>

In [ ]:
sixth_XGBoost = XGBoost.XGBClassifier(learning_rate = 0.01,
                                                  n_estimators = 449,
                                                  max_depth=3,
                                                  min_child_weight=1,
                                                  gamma=0.1,
                                                  subsample=0.6, # change this
                                                  colsample_bytree=0.6, #change this
                                                  objective= 'binary:logistic',
                                                  nthread=4,
                                                  scale_pos_weight=1,
                                                  reg_alpha = 0.1,
                                                  seed = 42) # then add reg_alpha

In [ ]:
xgb_param = sixth_XGBoost.get_xgb_params()
xgtrain = data_transform(reshaped_train_features, labels = train_label)
cvresult = XGBoost.cv(xgb_param, xgtrain, num_boost_round = sixth_XGBoost.get_params()['n_estimators'], nfold = 5,
            metrics='auc', early_stopping_rounds = 20)
sixth_XGBoost.set_params(n_estimators = cvresult.shape[0], eval_metric='auc')

#Fit the algorithm on the data
sixth_XGBoost.fit(reshaped_train_features, train_label)

print(cvresult.shape[0])

Finally, this is the end of the tuning algorithm and from the last algo, the best number of estimators are 449.

## <span style = "font-family: Cambria">Performance Testing</span>

In [ ]:
test_features = reshaped_test_features
test_labels = test_label

prediction = sixth_XGBoost.predict_proba(test_features)[:,1]

roc_auc = sklearn.metrics.roc_auc_score(test_labels, prediction)
fpr, tpr, _ = sklearn.metrics.roc_curve(test_labels, prediction)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
prediction_class = sixth_XGBoost.predict(test_features)
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(test_label, prediction_class).ravel()

# Calculate Precision, Recall, and Accuracy
precision = sklearn.metrics.precision_score(test_label, prediction_class)
accuracy = sklearn.metrics.accuracy_score(test_label, prediction_class)
recall = sklearn.metrics.recall_score(test_label, prediction_class)
f1_score = sklearn.metrics.f1_score(test_label, prediction_class)

print("F1_score:" ,f1_score)

# Calculate ROC AUC Score
auc = sklearn.metrics.roc_auc_score(test_label, prediction_class)


print("True Negatives:", tn)
print("False Positives:", fp)
print("False Negatives:", fn)
print("True Positives:", tp)
print("Precision:", precision)
print("Accuracy:", accuracy)
print("Recall:", recall)
print("AUC:", auc)

In [ ]:
XGBoost.plot_tree(sixth_XGBoost)

### <span style = "font-family: Cambria">Function to Perform all the hyper tuning above</span>

In [ ]:
def hyper_parameter_tuning(features, labels, n_jobs = 1, verbose = 0):

    model = XGBoost.XGBClassifier(learning_rate = 0.1,
                                n_estimators = 1000,
                                max_depth = 5,
                                min_child_weight = 1,
                                gamma = 0,
                                subsample = 0.8,
                                colsample_bytree = 0.8,
                                objective = 'binary:logistic',
                                nthread = 4,
                                scale_pos_weight = 1,
                                seed = 42)

    # Step 1.
    xgb_param = model.get_xgb_params()
    xgtrain = data_transform(features, labels = labels)
    cvresult = XGBoost.cv(xgb_param, xgtrain, num_boost_round = model.get_params()['n_estimators'], nfold = 5,
            metrics='auc', early_stopping_rounds = 20)

    model.set_params(n_estimators = cvresult.shape[0])

    print("Step 1 Done")
    print("Result: Best n_estimators = " , str(cvresult.shape[0]))

    # Step 2.
    param_grid_2 = {'max_depth':range(3,10,1),
                    'min_child_weight':range(1,6,1)
                   }
    grid_search_2 = GridSearchCV(model, param_grid = param_grid_2, scoring = 'roc_auc', n_jobs = n_jobs , cv = 5, verbose = verbose)
    grid_search_2.fit(features,labels)

    model.set_params(max_depth = grid_search_2.best_params_["max_depth"])
    model.set_params(min_child_weight = grid_search_2.best_params_["min_child_weight"])

    print("Step 2 Done")
    print("Result: Best max_depth, min_child_weight = " , str(grid_search_2.best_params_["max_depth"]) , "," ,
          str(grid_search_2.best_params_["min_child_weight"]))

    # Step 3.
    param_grid_3 = {'gamma':[i/10.0 for i in range(0,11)]}

    grid_search_3 = GridSearchCV(model, param_grid = param_grid_3, scoring = 'roc_auc', n_jobs = n_jobs , cv = 5, verbose = verbose)
    grid_search_3.fit(features,labels)

    model.set_params(gamma = grid_search_3.best_params_["gamma"])

    print("Step 3 Done")
    print("Result: Best gamma = " , str(grid_search_3.best_params_["gamma"]))

    # Step 4.
    param_grid_4 = {'subsample':[i/10.0 for i in range(5,11)],
                    'colsample_bytree':[i/10.0 for i in range(5,11)]}

    grid_search_4 = GridSearchCV(model, param_grid = param_grid_4, scoring = 'roc_auc', n_jobs = n_jobs , cv = 5, verbose = verbose)
    grid_search_4.fit(features,labels)

    model.set_params(subsample = grid_search_4.best_params_["subsample"])
    model.set_params(colsample_bytree = grid_search_4.best_params_["colsample_bytree"])

    print("Step 4 Done")
    print("Result: Best subsample, colsample_by_tree = " , str(grid_search_4.best_params_["subsample"]) , "," ,
          str(grid_search_4.best_params_["colsample_bytree"]))

    # Step 5.
    param_grid_5 = {'reg_alpha':[1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 10, 100]}

    grid_search_5 = GridSearchCV(model, param_grid = param_grid_5, scoring = 'roc_auc', n_jobs = n_jobs , cv = 5, verbose = verbose)
    grid_search_5.fit(features,labels)

    model.set_params(reg_alpha = grid_search_5.best_params_["reg_alpha"])

    print("Step 5 Done")
    print("Result: Best reg_alpha = " , str(grid_search_5.best_params_["reg_alpha"]))

    # Step 6.
    xgb_param = model.get_xgb_params()
    xgtrain = data_transform(features, labels = labels)
    cvresult = XGBoost.cv(xgb_param, xgtrain, num_boost_round = model.get_params()['n_estimators'], nfold = 5,
            metrics='auc', early_stopping_rounds = 20)
    model.set_params(n_estimators = cvresult.shape[0])

    model.fit(features, labels)

    print("Step 6 Done")
    print("Result: Best n_estimators = " , str(cvresult.shape[0]))

    return model

In [ ]:
model_pca_450 = hyper_parameter_tuning(reduced_features_450, train_label)

In [ ]:
model_pca_250 = hyper_parameter_tuning(reduced_features_250, train_label)

In [ ]:
test_features = test_features_450
test_labels = test_label

prediction = model_pca_450.predict_proba(test_features)[:,1]

roc_auc = sklearn.metrics.roc_auc_score(test_labels, prediction)
fpr, tpr, _ = sklearn.metrics.roc_curve(test_labels, prediction)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
prediction_class = model_pca_450.predict(test_features)
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(test_label, prediction_class).ravel()

# Calculate Precision, Recall, and Accuracy
precision = sklearn.metrics.precision_score(test_label, prediction_class)
accuracy = sklearn.metrics.accuracy_score(test_label, prediction_class)
recall = sklearn.metrics.recall_score(test_label, prediction_class)

# Calculate ROC AUC Score
auc = sklearn.metrics.roc_auc_score(test_label, prediction_class)

print("True Negatives:", tn)
print("False Positives:", fp)
print("False Negatives:", fn)
print("True Positives:", tp)
print("Precision:", precision)
print("Accuracy:", accuracy)
print("Recall:", recall)
print("AUC:", auc)

In [ ]:
test_features = test_features_250
test_labels = test_label

prediction = model_pca_250.predict_proba(test_features)[:,1]

roc_auc = sklearn.metrics.roc_auc_score(test_labels, prediction)
fpr, tpr, _ = sklearn.metrics.roc_curve(test_labels, prediction)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
prediction_class = model_pca_250.predict(test_features)
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(test_label, prediction_class).ravel()

# Calculate Precision, Recall, and Accuracy
precision = sklearn.metrics.precision_score(test_label, prediction_class)
accuracy = sklearn.metrics.accuracy_score(test_label, prediction_class)
recall = sklearn.metrics.recall_score(test_label, prediction_class)
f1_score = sklearn.metrics.f1_score(test_label, prediction_class)

print("F1_score:" ,f1_score)

# Calculate ROC AUC Score
auc = sklearn.metrics.roc_auc_score(test_label, prediction_class)

print("True Negatives:", tn)
print("False Positives:", fp)
print("False Negatives:", fn)
print("True Positives:", tp)
print("Precision:", precision)
print("Accuracy:", accuracy)
print("Recall:", recall)
print("AUC:", auc)